In [1]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVR, SVC
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score, silhouette_score

In [2]:
# Генерируем уникальный seed
my_code = "Bevz"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [3]:
# Данные загружены отсюда: https://www.kaggle.com/dwdkills/russian-demography
# Читаем данные из файла
example_data = pd.read_csv("../datasets/russian_demography.csv")

In [4]:
# "year" - год (1990-2017)
# "region" - название региона
# "npg" - естественный прирост населения на 1000 человек
# "birth_rate" - количество рождений на 1000 человек
# "death_rate" - количество смертей на 1000 человек
# "gdw" - коэффициент демографической нагрузки на 100 человек (Отношение числа нетрудоспособных к числу трудоспособных).
# "urbanization" - процент городского населения

example_data.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
0,1990,Republic of Adygea,1.9,14.2,12.3,84.66,52.42
1,1990,Altai Krai,1.8,12.9,11.1,80.24,58.07
2,1990,Amur Oblast,7.6,16.2,8.6,69.55,68.37
3,1990,Arkhangelsk Oblast,3.7,13.5,9.8,73.26,73.63
4,1990,Astrakhan Oblast,4.7,15.1,10.4,77.05,68.01


In [5]:
# Так как список регионов меняется от года к году, в данных есть строки без значений. Удалим их
example_data.dropna(inplace=True)

In [6]:
example_data.shape

(2315, 7)

In [7]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

463


In [8]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

1389 463 463


In [9]:
train.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
970,2001,Lipetsk Oblast,-9.1,8.0,17.1,72.08,64.09
2002,2013,Orenburg Oblast,0.9,14.8,13.9,68.23,59.70
77,1990,Khanty–Mansi Autonomous Okrug – Yugra,12.9,17.1,4.2,58.98,91.66
509,1995,Yaroslavl Oblast,-9.8,7.6,17.4,78.06,80.62
270,1993,Zabaykalsky Krai,-0.3,12.6,12.9,78.17,63.64


In [10]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = ['year', 'npg', 'birth_rate', 'death_rate', 'gdw', 'urbanization']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['year', 'npg', 'birth_rate', 'death_rate',
                                  'gdw', 'urbanization'])])

In [11]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [12]:
# Устанавливаем названия столбцов
column_names = columns_to_scale + ['region']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [13]:
sc_val

,year,npg,birth_rate,death_rate,gdw,urbanization,region
0,0.592593,0.355721,0.324895,0.571429,0.317017,0.528016,Zabaykalsky Krai
1,1.0,0.308458,0.198312,0.516129,0.789892,0.35267,Republic of Crimea
2,0.925926,0.437811,0.383966,0.479263,0.608942,0.721819,Irkutsk Oblast
3,0.518519,0.368159,0.206751,0.423963,0.10903,0.750824,Kamchatka Krai
4,0.444444,0.701493,0.43038,0.041475,0.664605,0.242848,Republic of Ingushetia
...,...,...,...,...,...,...,...
458,0.148148,0.233831,0.135021,0.585253,0.603994,0.64878,Arkhangelsk Oblast
459,0.185185,0.248756,0.118143,0.539171,0.666195,0.753856,Chelyabinsk Oblast
460,0.296296,0.380597,0.092827,0.267281,0.23096,0.894792,Murmansk Oblast
461,1.0,0.390547,0.261603,0.43318,0.744301,0.592617,Republic of Khakassia


In [14]:
# Вспоминаем алгоритмы решения задачи регрессии
r_models = []

# Линейная регрессия
# Для использования регуляризации, вместо LinearRegression используем Lasso, Ridge или ElasticNet
# Параметр alpha - коэффициент регуляризации для Lasso и Ridge, по умолчанию равен 1
# Для ElasticNet, если регуляризация иммет вид a*L1+b*L2, то
# параметр alpha = a + b, по умолчанию равен 1
# параметр l1_ratio = a / (a + b), по умолчанию равен 0.5
r_models.append(LinearRegression())
r_models.append(Lasso(alpha=1.0))
r_models.append(Ridge(alpha=1.0))
r_models.append(ElasticNet(alpha=1.0, l1_ratio=0.5))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
r_models.append(KNeighborsRegressor(n_neighbors=5))
r_models.append(KNeighborsRegressor(n_neighbors=10))
r_models.append(KNeighborsRegressor(n_neighbors=15))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
r_models.append(SVR(kernel='linear'))
r_models.append(SVR(kernel='poly'))
r_models.append(SVR(kernel='rbf'))
r_models.append(SVR(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'mse', 'friedman_mse', 'mae', 'poisson'
r_models.append(DecisionTreeRegressor(criterion='squared_error'))
r_models.append(DecisionTreeRegressor(criterion='friedman_mse'))
r_models.append(DecisionTreeRegressor(criterion='absolute_error'))

In [15]:
r_models

[LinearRegression(),
 Lasso(),
 Ridge(),
 ElasticNet(),
 KNeighborsRegressor(),
 KNeighborsRegressor(n_neighbors=10),
 KNeighborsRegressor(n_neighbors=15),
 SVR(kernel='linear'),
 SVR(kernel='poly'),
 SVR(),
 SVR(kernel='sigmoid'),
 DecisionTreeRegressor(),
 DecisionTreeRegressor(criterion='friedman_mse'),
 DecisionTreeRegressor(criterion='absolute_error')]

In [16]:
# Выделим предикторы и зависимую переменную
x_labels = column_names[0:-2]
y_labels = ['urbanization']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [17]:
x_labels

['year', 'npg', 'birth_rate', 'death_rate', 'gdw']

In [18]:
y_labels

['urbanization']

In [19]:
# Обучаем модели
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

LinearRegression()
Lasso()
Ridge()
ElasticNet()
KNeighborsRegressor()
KNeighborsRegressor(n_neighbors=10)
KNeighborsRegressor(n_neighbors=15)
SVR(kernel='linear')
SVR(kernel='poly')
SVR()
SVR(kernel='sigmoid')
DecisionTreeRegressor()
DecisionTreeRegressor(criterion='friedman_mse')
DecisionTreeRegressor(criterion='absolute_error')


In [20]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

LinearRegression() 	 0.02070899447283525
Lasso() 	 0.029875590195645473
Ridge() 	 0.02077849722675898
ElasticNet() 	 0.029875590195645473
KNeighborsRegressor() 	 0.010643372888868732
KNeighborsRegressor(n_neighbors=10) 	 0.010794644343593982
KNeighborsRegressor(n_neighbors=15) 	 0.012364135940286201
SVR(kernel='linear') 	 0.02090201311708424
SVR(kernel='poly') 	 0.013593928578079657
SVR() 	 0.01197804508027608
SVR(kernel='sigmoid') 	 31.43500093605669
DecisionTreeRegressor() 	 0.018515645650358105
DecisionTreeRegressor(criterion='friedman_mse') 	 0.01887044840294862
DecisionTreeRegressor(criterion='absolute_error') 	 0.021825672200254657


In [21]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

KNeighborsRegressor()

In [22]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.011226003386042998


In [23]:
# Вспоминаем алгоритмы решения задачи классификации
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='none', solver='saga', max_iter=500))
c_models.append(LogisticRegression(penalty='l1', solver='saga', max_iter=500))
c_models.append(LogisticRegression(penalty='l2', solver='saga'))
c_models.append(LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'))
c_models.append(LogisticRegression())

# Наивный байесовский классификатор
# Параметр alpha - параметр сглаживания, по умолчанию равен 1 (сглаживание Лапласа)
c_models.append(MultinomialNB(alpha=0.0))
c_models.append(MultinomialNB(alpha=0.5))
c_models.append(MultinomialNB(alpha=1.0))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))
c_models.append(KNeighborsClassifier(n_neighbors=10))
c_models.append(KNeighborsClassifier(n_neighbors=15))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
c_models.append(SVC(kernel='linear'))
c_models.append(SVC(kernel='poly'))
c_models.append(SVC(kernel='rbf'))
c_models.append(SVC(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'gini', 'entropy'
# Параметр splitter - стартегия ветвления: 'best', 'random'
c_models.append(DecisionTreeClassifier(criterion='gini', splitter='best'))
c_models.append(DecisionTreeClassifier(criterion='gini', splitter='random'))
c_models.append(DecisionTreeClassifier(criterion='entropy', splitter='best'))
c_models.append(DecisionTreeClassifier(criterion='entropy', splitter='random'))

In [24]:
c_models

[LogisticRegression(max_iter=500, penalty='none', solver='saga'),
 LogisticRegression(max_iter=500, penalty='l1', solver='saga'),
 LogisticRegression(solver='saga'),
 LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga'),
 LogisticRegression(),
 MultinomialNB(alpha=0.0),
 MultinomialNB(alpha=0.5),
 MultinomialNB(),
 KNeighborsClassifier(),
 KNeighborsClassifier(n_neighbors=10),
 KNeighborsClassifier(n_neighbors=15),
 SVC(kernel='linear'),
 SVC(kernel='poly'),
 SVC(),
 SVC(kernel='sigmoid'),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(splitter='random'),
 DecisionTreeClassifier(criterion='entropy'),
 DecisionTreeClassifier(criterion='entropy', splitter='random')]

In [25]:
# Выделим предикторы и метки классов
x_labels = column_names[0:-1]
y_labels = ['region']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [26]:
x_train

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.407407,0.169154,0.075949,0.640553,0.594098,0.526566
1,0.851852,0.41791,0.362869,0.493088,0.526065,0.468688
2,0.0,0.716418,0.459916,0.046083,0.362608,0.890046
3,0.185185,0.151741,0.059072,0.654378,0.69977,0.744496
4,0.111111,0.38806,0.270042,0.447005,0.701714,0.520633
...,...,...,...,...,...,...
1384,0.814815,0.728856,0.544304,0.115207,0.393179,0.276203
1385,0.185185,0.226368,0.088608,0.548387,0.708429,0.522874
1386,0.037037,0.587065,0.506329,0.336406,0.910408,0.038497
1387,0.333333,0.19403,0.075949,0.59447,0.620604,0.591299


In [27]:
# Обучаем модели
for model in c_models:
    print(model)
    model.fit(x_train, y_train)

LogisticRegression(max_iter=500, penalty='none', solver='saga')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(max_iter=500, penalty='l1', solver='saga')


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


LogisticRegression(solver='saga')
LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga')
LogisticRegression()
MultinomialNB(alpha=0.0)
MultinomialNB(alpha=0.5)


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/naive_bayes.py:557: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  % _ALPHA_MIN


MultinomialNB()
KNeighborsClassifier()
KNeighborsClassifier(n_neighbors=10)
KNeighborsClassifier(n_neighbors=15)
SVC(kernel='linear')
SVC(kernel='poly')
SVC()
SVC(kernel='sigmoid')
DecisionTreeClassifier()
DecisionTreeClassifier(splitter='random')
DecisionTreeClassifier(criterion='entropy')
DecisionTreeClassifier(criterion='entropy', splitter='random')


In [28]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(model, '\t', f1)

LogisticRegression(max_iter=500, penalty='none', solver='saga') 	 0.38740604497553727
LogisticRegression(max_iter=500, penalty='l1', solver='saga') 	 0.13042062569822593
LogisticRegression(solver='saga') 	 0.04364613532275361
LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga') 	 0.051331459873476204
LogisticRegression() 	 0.043444046226723404
MultinomialNB(alpha=0.0) 	 0.019040817594235645
MultinomialNB(alpha=0.5) 	 0.01865448761690914
MultinomialNB() 	 0.018605044746135044
KNeighborsClassifier() 	 0.41024481736589474
KNeighborsClassifier(n_neighbors=10) 	 0.2808886561094518
KNeighborsClassifier(n_neighbors=15) 	 0.23288234540393565
SVC(kernel='linear') 	 0.06202549510793355
SVC(kernel='poly') 	 0.6781433059811719
SVC() 	 0.19676147822109555
SVC(kernel='sigmoid') 	 9.309600059581442e-06
DecisionTreeClassifier() 	 0.638328378891338
DecisionTreeClassifier(splitter='random') 	 0.5861837049825603
DecisionTreeClassifier(criterion='entropy') 	 0.6462840660941284
DecisionTre

In [29]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = c_models[i_max]
best_c_model

SVC(kernel='poly')

In [30]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.7097044406630229


In [31]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=50))

In [32]:
# Выделим используемые параметры
x_labels = column_names[0:-1]
x = pd.concat([sc_train[x_labels], sc_val[x_labels], sc_test[x_labels]])
x

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.407407,0.169154,0.075949,0.640553,0.594098,0.526566
1,0.851852,0.41791,0.362869,0.493088,0.526065,0.468688
2,0.0,0.716418,0.459916,0.046083,0.362608,0.890046
3,0.185185,0.151741,0.059072,0.654378,0.69977,0.744496
4,0.111111,0.38806,0.270042,0.447005,0.701714,0.520633
...,...,...,...,...,...,...
458,0.37037,0.159204,0.054852,0.635945,0.598869,0.533289
459,0.037037,0.753731,0.746835,0.290323,0.802085,0.306262
460,0.481481,0.671642,0.367089,0.032258,0.699063,0.241266
461,0.851852,0.343284,0.329114,0.59447,0.68581,0.481872


In [33]:
# Произведем кластеризацию
for model in k_models:
    model.fit(x)

In [34]:
# Оценим качество результата
sils = []
for model in k_models:
    cluster_labels = model.predict(x)
    s = silhouette_score(x, cluster_labels)
    sils.append(s)
    print(s)

0.2949840478420507
0.2528801801932595
0.23512376680134522
0.23167525334869754


In [35]:
# Выбираем лучшую модель
i_max = sils.index(max(sils))
best_k_model = k_models[i_max]
print(best_k_model)
print(sils[i_max])

KMeans(n_clusters=5)
0.2949840478420507


In [36]:
# Задание №1 - анализ моделей для задачи регрессии
# Общий список моделей
r_models = [
    LinearRegression(),
    Lasso(alpha=1.0),
    Lasso(alpha=0.5),
    Ridge(alpha=1.0),
    Ridge(alpha=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.25),
    ElasticNet(alpha=1.0, l1_ratio=0.75),
    ElasticNet(alpha=0.5, l1_ratio=0.5),
    ElasticNet(alpha=0.5, l1_ratio=0.25),
    ElasticNet(alpha=0.5, l1_ratio=0.75),
    KNeighborsRegressor(n_neighbors=5),
    KNeighborsRegressor(n_neighbors=10),
    KNeighborsRegressor(n_neighbors=15),
    KNeighborsRegressor(n_neighbors=20),
    KNeighborsRegressor(n_neighbors=25),
    SVR(kernel='linear'),
    SVR(kernel='poly'),
    SVR(kernel='rbf'),
    SVR(kernel='sigmoid'),
    DecisionTreeRegressor(criterion='squared_error'),
    DecisionTreeRegressor(criterion='friedman_mse'),
    DecisionTreeRegressor(criterion='absolute_error')
]

In [37]:
# Выбор моделей для задания
n = 4
random.seed(my_seed)
my_models1 = random.sample(r_models, n)
print(my_models1)

[KNeighborsRegressor(n_neighbors=25), SVR(), ElasticNet(l1_ratio=0.75), ElasticNet(alpha=0.5, l1_ratio=0.75)]


In [38]:
# Загрузим данные для задачи регрессии
data = pd.read_csv("../datasets/weather.csv", decimal=',')
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change
0,468.0,0.0,24.9,31.0,100.0,4.0,0.0,993.8,4.0,-3.0,-0.3,-0.4
1,140.0,0.0,27.9,22.0,20.0,1.0,0.0,1004.1,4.0,-2.0,0.6,-1.0
2,464.0,0.0,25.6,28.0,20.0,2.0,1.0,1001.2,4.0,-7.0,-1.8,-0.3
3,467.0,0.0,26.7,26.0,100.0,3.0,0.0,992.3,4.0,-1.0,1.2,-1.3
4,138.0,0.0,29.1,22.0,20.0,1.0,0.0,1001.3,4.0,-2.0,2.5,-1.3
...,...,...,...,...,...,...,...,...,...,...,...,...
1434,474.0,0.1,6.4,95.0,4.0,1.0,3.0,999.9,4.0,-1.0,5.9,1.5
1435,65.0,0.0,6.4,93.0,10.0,4.0,3.0,983.9,4.0,-1.0,5.2,3.1
1436,159.0,0.1,8.5,83.0,10.0,2.0,1.0,969.7,4.0,1.0,1.4,1.9
1437,99.0,0.1,7.5,91.0,10.0,3.0,3.0,995.4,4.0,-4.0,2.7,1.7


In [39]:
# Зависимая переменная для всех одна и та же. Предикторы выбираем случайнм образом.
columns = list(data.columns)
n_x = 5

y_label = 'water_level'
x_labels = random.sample(columns[1:], n_x)

print(x_labels)

['temp_change', 'wind', 'fire', 'precipitation', 'visibility']


In [40]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу регрессии с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

data = data.drop(['temperature', 'weather', 'pressure', 'fire', 'temp_change', 'pressure_change'],axis=1)

In [41]:
data

,water_level,precipitation,humidity,visibility,wind,wl_change
0,468.0,0.0,31.0,100.0,4.0,-3.0
1,140.0,0.0,22.0,20.0,1.0,-2.0
2,464.0,0.0,28.0,20.0,2.0,-7.0
3,467.0,0.0,26.0,100.0,3.0,-1.0
4,138.0,0.0,22.0,20.0,1.0,-2.0
...,...,...,...,...,...,...
1434,474.0,0.1,95.0,4.0,1.0,-1.0
1435,65.0,0.0,93.0,10.0,4.0,-1.0
1436,159.0,0.1,83.0,10.0,2.0,1.0
1437,99.0,0.1,91.0,10.0,3.0,-4.0


In [42]:
data.shape
val_test_size = round(0.2*len(data))
print(val_test_size)

288


In [43]:
random_state = my_seed
train_val, test = train_test_split(data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

863 288 288


In [44]:
train.head()

,water_level,precipitation,humidity,visibility,wind,wl_change
1193,137.0,0.0,68.0,20.0,4.0,-3.0
1295,133.0,0.0,72.0,20.0,1.0,-3.0
1274,145.0,0.0,89.0,100.0,3.0,-4.0
323,177.0,0.0,73.0,20.0,2.0,-2.0
615,497.0,0.0,79.0,20.0,2.0,30.0


In [45]:
columns_to_scale = ['water_level', 'precipitation', 'wl_change', 'humidity', 'visibility', 'wind']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['water_level', 'precipitation', 'wl_change',
                                  'humidity', 'visibility', 'wind'])])

In [46]:
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [47]:
column_names = columns_to_scale 
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [48]:
sc_val

,water_level,precipitation,wl_change,humidity,visibility,wind
0,0.335489,0.0,0.395,0.780220,0.10,0.1
1,0.013743,0.0,0.420,0.439560,0.20,0.0
2,0.236055,0.0,0.405,0.813187,0.20,0.2
3,0.543250,0.0,0.450,0.208791,0.10,0.1
4,0.605497,0.0,0.385,0.912088,0.04,0.1
...,...,...,...,...,...,...
283,0.242522,0.0,0.395,0.571429,0.20,0.1
284,0.162490,0.0,0.400,0.604396,0.20,0.3
285,0.734842,0.0,0.385,0.329670,1.00,0.1
286,0.490703,0.0,0.315,0.131868,0.10,0.3


In [49]:
r_models = []

r_models.append(ElasticNet(alpha=0.5))
r_models.append(ElasticNet(alpha=0.5, l1_ratio=0.75))

r_models.append(SVR())

r_models.append(Ridge())
r_models

[ElasticNet(alpha=0.5), ElasticNet(alpha=0.5, l1_ratio=0.75), SVR(), Ridge()]

In [50]:
x_labels = column_names[0:-1]
y_labels = ['water_level']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [51]:
for model in r_models:
    print(model)
    model.fit(x_train, y_train)

ElasticNet(alpha=0.5)
ElasticNet(alpha=0.5, l1_ratio=0.75)
SVR()
Ridge()


In [52]:
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(model, '\t', mse)

ElasticNet(alpha=0.5) 	 0.03391137170911462
ElasticNet(alpha=0.5, l1_ratio=0.75) 	 0.03391137170911462
SVR() 	 0.001736505487514592
Ridge() 	 5.0871865186483215e-05


In [53]:
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

Ridge()

In [54]:
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

5.0807632836659245e-05


In [55]:
# Задание №2 - анализ моделей для задачи классификации
# Общий список моделей
c_models = [
    LogisticRegression(penalty='none', solver='saga'),
    LogisticRegression(penalty='l1', solver='saga'),
    LogisticRegression(penalty='l2', solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.25, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.75, solver='saga'),
    LogisticRegression(),
    MultinomialNB(alpha=0.0),
    MultinomialNB(alpha=0.25),
    MultinomialNB(alpha=0.5),
    MultinomialNB(alpha=0.75),
    MultinomialNB(alpha=1.0),
    KNeighborsClassifier(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=10),
    KNeighborsClassifier(n_neighbors=15),
    KNeighborsClassifier(n_neighbors=20),
    KNeighborsClassifier(n_neighbors=25),
    SVC(kernel='linear'),
    SVC(kernel='poly'),
    SVC(kernel='rbf'),
    SVC(kernel='sigmoid'),
    DecisionTreeClassifier(criterion='gini', splitter='best'),
    DecisionTreeClassifier(criterion='gini', splitter='random'),
    DecisionTreeClassifier(criterion='entropy', splitter='best'),
    DecisionTreeClassifier(criterion='entropy', splitter='random')
]

In [56]:
# Выбор моделей для задания
n = 5
my_models2 = random.sample(c_models, n)
print(my_models2)

[SVC(), MultinomialNB(alpha=0.25), SVC(kernel='poly'), DecisionTreeClassifier(splitter='random'), LogisticRegression()]


In [57]:
# Загрузим данные для задачи классификации
newdata = pd.read_csv("../datasets/zoo2.csv")
newdata

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,turtle,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,chameleon,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,iguana,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,lizard,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,gecko,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
5,python,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
6,boa,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,1,3
7,adder,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
8,crocodile,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3
9,alligator,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3


In [58]:
# Метка класса для всех одна и та же. Параметры выбираем случайнм образом.
columns = list(newdata.columns)
n_x = 8

y_newlabel = 'class_type'
x_newlabels = random.sample(columns[1:-1], n_x)

print(x_newlabels)

['predator', 'aquatic', 'domestic', 'milk', 'legs', 'breathes', 'fins', 'venomous']


In [59]:
labels = x_newlabels.copy()
labels.append(y_newlabel)
temp_data = newdata[labels]
sec_data = temp_data

In [60]:
sec_data

,predator,aquatic,domestic,milk,legs,breathes,fins,venomous,class_type
0,0,1,1,0,4,1,0,0,3
1,0,0,1,0,4,1,0,0,3
2,1,0,1,0,4,1,0,0,3
3,1,0,0,0,4,1,0,0,3
4,0,0,1,0,4,1,0,0,3
5,1,0,0,0,0,1,0,1,3
6,1,0,0,0,0,1,0,0,3
7,1,0,0,0,0,1,0,1,3
8,1,1,0,0,4,1,0,0,3
9,1,1,0,0,4,1,0,0,3


In [61]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу классификации с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

sec_data.shape

(43, 9)

In [62]:
val_test_size2 = round(0.2*len(sec_data))
print(val_test_size2)

9


In [63]:
random_state = my_seed
train_val2, test2 = train_test_split(sec_data, test_size=val_test_size2, random_state=random_state)
sec_train, val2 = train_test_split(train_val2, test_size=val_test_size2, random_state=random_state)
print(len(sec_train), len(val2), len(test2))

25 9 9


In [64]:
sec_train.head()

,predator,aquatic,domestic,milk,legs,breathes,fins,venomous,class_type
23,0,1,0,0,4,1,0,0,5
19,0,1,1,0,4,1,0,0,5
10,1,1,0,0,4,1,0,0,3
14,0,1,0,0,0,0,1,0,4
39,0,0,0,0,0,1,0,0,7


In [65]:
columns_to_scale2 = x_newlabels

ct2 = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale2)], remainder='passthrough')
ct2.fit(sec_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['predator', 'aquatic', 'domestic', 'milk',
                                  'legs', 'breathes', 'fins', 'venomous'])])

In [66]:
sc_train2 = pd.DataFrame(ct2.transform(sec_train))
sc_test2 = pd.DataFrame(ct2.transform(test2))
sc_val2 = pd.DataFrame(ct2.transform(val2))
column_names2 = columns_to_scale2 + ['class_type']
sc_train2.columns = column_names2
sc_test2.columns = column_names2
sc_val2.columns = column_names2
sc_val2

,predator,aquatic,domestic,milk,legs,breathes,fins,venomous,class_type
0,0.0,0.0,0.0,0.0,1.000000,1.0,0.0,0.0,6.0
1,1.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0,3.0
2,1.0,0.0,0.0,0.0,0.666667,1.0,0.0,0.0,3.0
3,0.0,0.0,0.0,0.0,1.000000,1.0,0.0,0.0,6.0
4,0.0,0.0,1.0,0.0,0.666667,1.0,0.0,0.0,3.0
5,0.0,0.0,0.0,0.0,0.666667,1.0,0.0,0.0,3.0
6,1.0,0.0,0.0,0.0,1.333333,1.0,0.0,1.0,7.0
7,0.0,1.0,0.0,0.0,0.666667,1.0,0.0,0.0,5.0
8,1.0,1.0,0.0,0.0,0.000000,0.0,1.0,0.0,4.0


In [67]:
# Вспоминаем алгоритмы решения задачи классификации
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='l1', solver='saga'))


# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))

# Метод опорных векторов
# Параметр kernel опредеяет вид ядра преобразования
c_models.append(SVC(kernel='sigmoid'))

# Деревья решений
# Параметр criterion - критерий качества ветвления: 'gini', 'entropy'
# Параметр splitter - стартегия ветвления: 'best', 'random'
c_models.append(DecisionTreeClassifier())
c_models.append(DecisionTreeClassifier(splitter='random'))
c_models
[LogisticRegression(penalty='l1', solver='saga'),
 KNeighborsClassifier(),
 SVC(kernel='sigmoid'),
 DecisionTreeClassifier(),
 DecisionTreeClassifier(splitter='random')]
x_labels2 = column_names2[0:-1]
y_labels2 = ['class_type']

x_train2 = sc_train2[x_labels2]
x_test2 = sc_test2[x_labels2]
x_val2 = sc_val2[x_labels2]

y_train2 = np.ravel(sc_train2[y_labels2])
y_test2 = np.ravel(sc_test2[y_labels2])
y_val2 = np.ravel(sc_val2[y_labels2])
x_train2

,predator,aquatic,domestic,milk,legs,breathes,fins,venomous
0,0.0,1.0,0.0,0.0,0.666667,1.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.666667,1.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.666667,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.000000,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0
5,1.0,1.0,0.0,0.0,0.666667,1.0,0.0,0.0
6,0.0,1.0,0.0,0.0,0.000000,0.0,1.0,0.0
7,0.0,1.0,0.0,0.0,0.000000,0.0,1.0,0.0
8,1.0,0.0,1.0,0.0,0.666667,1.0,0.0,0.0
9,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [68]:
# Обучаем модели
for model2 in c_models:
    print(model2)
    model2.fit(x_train2, y_train2)

LogisticRegression(penalty='l1', solver='saga')
KNeighborsClassifier()
SVC(kernel='sigmoid')
DecisionTreeClassifier()
DecisionTreeClassifier(splitter='random')


In [69]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model2 in c_models:
    val_pred2 = model2.predict(x_val2)
    f1 = f1_score(y_val2, val_pred2, average='weighted')
    f1s.append(f1)
    print(model2, '\t', f1)

LogisticRegression(penalty='l1', solver='saga') 	 0.6455026455026455
KNeighborsClassifier() 	 0.7513227513227513
SVC(kernel='sigmoid') 	 0.48148148148148145
DecisionTreeClassifier() 	 0.6839506172839506
DecisionTreeClassifier(splitter='random') 	 0.8444444444444444


In [70]:
# Выбираем лучшую модель
i_max = f1s.index(max(f1s))
best_c_model = c_models[i_max]
best_c_model

DecisionTreeClassifier(splitter='random')

In [71]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred2 = best_c_model.predict(x_test2)
f1 = f1_score(y_test2, test_pred2, average='weighted')
print(f1)

0.8444444444444444


In [101]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=19))

In [102]:
# Выделим используемые параметры
x_labels2 = column_names2[0:-1]
del(x)
x = pd.concat([sc_train2[x_labels2], sc_val2[x_labels2], sc_test2[x_labels2]])
x

,predator,aquatic,domestic,milk,legs,breathes,fins,venomous
0,0.0,1.0,0.0,0.0,0.666667,1.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.666667,1.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.666667,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.000000,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,0.0
5,1.0,1.0,0.0,0.0,0.666667,1.0,0.0,0.0
6,0.0,1.0,0.0,0.0,0.000000,0.0,1.0,0.0
7,0.0,1.0,0.0,0.0,0.000000,0.0,1.0,0.0
8,1.0,0.0,1.0,0.0,0.666667,1.0,0.0,0.0
9,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [103]:
# Произведем кластеризацию
for model in k_models:
    model.fit(x)

In [104]:
# Оценим качество результата
sils = []
for model2 in k_models:
    cluster_labels = model2.predict(x)
    s = silhouette_score(x, cluster_labels)
    sils.append(s)
    print(s)

0.49965061510523007
0.6083434171777852
0.7906976709532183


In [105]:
# Выбираем лучшую модель
i_max = sils.index(max(sils))
best_k_model = k_models[i_max]
print(best_k_model)
print(sils[i_max])

KMeans(n_clusters=19)
0.7906976709532183
